# Add Github Pull Request comments 

In [32]:
# nuclio: ignore
import nuclio

In [33]:
%nuclio config spec.image = "mlrun/mlrun"
%nuclio config kind = "job"

%nuclio: setting spec.image to 'mlrun/mlrun'
%nuclio: setting kind to 'job'


In [34]:
import requests
import os
from mlrun import DataItem, get_run_db

def pr_comment(context, repo: str, issue : int,
               message: str = '', 
               message_file: DataItem = None):

    token = context.get_secret('GITHUB_TOKEN') or os.environ.get('GITHUB_TOKEN')
    if message_file and not message:
        message = message_file.get()
    elif not message and not message_file:
        raise ValueError('pr message or message file must be provided')
        
    headers = {'Accept': 'application/vnd.github.v3+json',
               'Authorization': f'token {token}'}
    url = f'https://api.github.com/repos/{repo}/issues/{issue}/comments'

    resp = requests.post(url=url, json={"body": str(message)}, headers=headers)
    if not resp.ok:
        errmsg = f'bad pr comment resp!!\n{resp.text}'
        context.logger.error(errmsg)
        raise IOError(errmsg)
    
def run_summary_comment(context, project, workflow_id, repo: str, issue : int):
    if not repo:
        print('repo not defined')
        return
    
    db = get_run_db().connect()
    html = db.list_runs(project=project, labels=f'workflow={workflow_id}').show(display=False)
    pr_comment(context, repo, issue, html)
    

In [5]:
# nuclio: end-code
# marks the end of a code section

In [35]:
from mlrun import mlconf
import os

mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

In [36]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("github_utils")

# add metadata (for templates and reuse)
fn.spec.default_handler = "run_summary_comment"
fn.spec.description = "add comments to github pull requests"
fn.metadata.categories = ["notifications", "utils"]
fn.metadata.labels = {"author": "yaronh"}
fn.export("function.yaml")

[mlrun] 2020-05-09 23:29:49,366 function spec saved to path: function.yaml


## tests

In [31]:
from mlrun import run_local, NewTask

token = "<your github token>"
task = NewTask('git-comment', handler=pr_comment)
task.with_params(repo='yaronha/tstactions', issue=2, message='xyz123')
task.with_secrets('inline', {'git_token': token})

run_local(task)


[mlrun] 2020-05-07 23:45:41,197 artifact path is not defined or is local, artifacts will not be visible in the UI
[mlrun] 2020-05-07 23:45:41,204 starting run git-comment uid=c394b499a69a423f876d4c028b2e29fe  -> http://10.199.227.162:8080


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...8b2e29fe,0,May 07 23:45:41,completed,git-comment,v3io_user=adminkind=handlerowner=adminhost=jupyter-65887d7ffb-5jsn2,,repo=yaronha/tstactionsissue=2message=xyz123,id=625552127,


to track results use .show() or .logs() or in CLI: 
!mlrun get run c394b499a69a423f876d4c028b2e29fe --project default , !mlrun logs c394b499a69a423f876d4c028b2e29fe --project default
[mlrun] 2020-05-07 23:45:41,794 run executed, status=completed
